已将项目根目录添加到Python路径: D:\App\Pycharm\1009MovieSchedulingSimulation



✅ 所有模块已成功导入！


In [1]:
import sys
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import matplotlib

# --- [ 步骤 0: 环境与显示设置 ] ---
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['axes.unicode_minus'] = False
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)


In [2]:
# --- [ 步骤 1: 数据加载与预处理 ] ---
print("\n--- 1. 正在加载并预处理评估数据 ---")
csv_path = '../logs/full_evaluation_results.csv'
try:
    df = pd.read_csv(csv_path)
    df['daily_actions'] = df['daily_actions'].apply(json.loads)
    df['daily_incomes'] = df['daily_incomes'].apply(json.loads)
    print(f"✅ 数据加载成功，共 {len(df)} 条模拟记录。")
except FileNotFoundError:
    print(f"❌ 错误：在路径 '{csv_path}' 下找不到结果文件。")
    print("👉 请先确保您已经成功运行了最新的评估脚本 main.py。")
    df = pd.DataFrame()


--- 1. 正在加载并预处理评估数据 ---
✅ 数据加载成功，共 132 条模拟记录。


In [3]:
import numpy as np
import pandas as pd

# (假设 df 变量已经从你之前的步骤加载完毕)

# --- [ 步骤 2: 定义所有分析函数 ] ---
def calculate_sharpe(daily_rewards):
    """计算夏普比率"""
    # 【修改】: 确保 daily_incomes 被正确解析
    # 检查 daily_rewards 是否为字符串（从 CSV 读入时）
    if isinstance(daily_rewards, str):
        try:
            # 假设它是JSON字符串
            daily_rewards = pd.read_json(daily_rewards, typ='series')
        except ValueError:
            return np.nan # 如果解析失败，返回NaN
            
    daily_rewards = np.asarray(daily_rewards)
    
    # 检查标准差是否为0
    std_dev = daily_rewards.std()
    if std_dev == 0 or np.isnan(std_dev):
        return 0.0
        
    return daily_rewards.mean() / std_dev

# --- [ 步骤 3: 计算并展示高级性能与风险指标 (已合并场景) ] ---
if not df.empty:
    print("\n" + "="*80)
    print("### 核心性能与风险稳定性指标分析 (所有日期汇总) ###")
    print("="*80)
    
    # 【修改】: 不再按 'scenario' 分组
    summary_avg = df.groupby('policy')['total_income'].mean()
    
    print("\n--- 核心性能指标：性能提升百分比 ---\n")
    
    sac_policy_name = 'RL (SAC)'
    if sac_policy_name not in summary_avg.index:
        print(f"警告：在结果中找不到主模型 '{sac_policy_name}'，跳过性能提升计算。")
    else:
        sac_income = summary_avg[sac_policy_name]
        
        # 查找包含 'SAC' 的所有策略名
        sac_variants_to_drop = summary_avg.index[summary_avg.index.str.contains('SAC')]
        # 排除所有 'SAC' 变体，剩下的都是基准策略
        baselines_incomes = summary_avg.drop(sac_variants_to_drop)
        
        if not baselines_incomes.empty:
            print(f"在【所有日期】汇总场景下:")
            print(f"  - {sac_policy_name} 平均收入: {sac_income:.2f} 万元")
            
            # ========================= 【【修改点：循环对比所有基准】】 =========================
            print("\n  --- SAC 与各基准策略对比 ---")
            for baseline_name, baseline_income in baselines_incomes.items():
                if baseline_income > 0:
                    improvement_pct = (sac_income / baseline_income - 1) * 100
                    print(f"    - 相对于 {baseline_name:<10} ({baseline_income:.2f} 万): {improvement_pct:+.2f}%")
                else:
                    print(f"    - 无法对比 {baseline_name:<10} (基准收入为0)")
            
            # (保留最佳基准的对比)
            best_baseline_income = baselines_incomes.max()
            best_baseline_name = baselines_incomes.idxmax()
            total_improvement_pct = (sac_income / best_baseline_income - 1) * 100
            print(f"\n  - 最佳基准策略 ({best_baseline_name}): {best_baseline_income:.2f} 万元")
            print(f"  - SAC 性能提升百分比 (相较最佳基准): {total_improvement_pct:.2f}%\n")
            # =================================================================================

        else:
            print(f"在【所有日期】汇总场景下: 没有找到可供对比的基准策略。\n")

    # 【【修改】】: 不再按 'scenario' 分组
    summary_min = df.groupby('policy')['total_income'].min().to_frame(name='最差情况表现 (万元)')
    print("\n--- 风险指标：最差情况表现 (万元) ---\n数值越大，抗风险能力越强：")
    print(summary_min.round(2))

    # 【【修改】】: 确保 daily_incomes 被正确解析
    # 假设 'daily_incomes' 是从 CSV 读入的 JSON 字符串
    df['sharpe_ratio'] = df['daily_incomes'].apply(calculate_sharpe)
    
    # 【【修改】】: 不再按 'scenario' 分组
    summary_sharpe = df.groupby('policy')['sharpe_ratio'].mean().to_frame(name='平均夏普比率')
    print("\n--- 风险指标：平均夏普比率 ---\n数值越大，风险调整后收益越高：")
    print(summary_sharpe.round(2))


### 核心性能与风险稳定性指标分析 (所有日期汇总) ###

--- 核心性能指标：性能提升百分比 ---

在【所有日期】汇总场景下:
  - RL (SAC) 平均收入: 77.37 万元

  --- SAC 与各基准策略对比 ---
    - 相对于 效率启发式策略    (59.06 万): +31.02%
    - 相对于 贪婪启发式策略    (49.24 万): +57.14%
    - 相对于 静态启发式策略    (31.06 万): +149.08%

  - 最佳基准策略 (效率启发式策略): 59.06 万元
  - SAC 性能提升百分比 (相较最佳基准): 31.02%


--- 风险指标：最差情况表现 (万元) ---
数值越大，抗风险能力越强：
          最差情况表现 (万元)
policy               
RL (SAC)        18.68
效率启发式策略         20.24
贪婪启发式策略         17.37
静态启发式策略         13.36

--- 风险指标：平均夏普比率 ---
数值越大，风险调整后收益越高：
          平均夏普比率
policy          
RL (SAC)    1.17
效率启发式策略     1.45
贪婪启发式策略     1.26
静态启发式策略     1.41


In [5]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import os
import json
import matplotlib.pyplot as plt
import sys

# 设置绘图字体，防止中文乱码
# Windows通常使用 SimHei 或 Microsoft YaHei
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

# -----------------------------------------------------------------------------
# 1. 数据加载函数
# -----------------------------------------------------------------------------
def load_data(csv_path='logs/full_evaluation_results.csv'):
    """读取CSV并解析JSON列"""
    
    # --- [关键修复] 智能寻找绝对路径 (兼容 Jupyter/控制台) ---
    try:
        # 正常脚本运行时使用 __file__
        current_dir = os.path.dirname(os.path.abspath(__file__))
    except NameError:
        # Jupyter Notebook 或 Python Console 中 __file__ 未定义
        # 使用当前工作目录代替
        current_dir = os.getcwd()
        print(f"⚠️ 检测到交互式环境，使用当前工作目录: {current_dir}")

    # 假设项目结构是 standard 的，向上找或者就在当前
    possible_paths = [
        csv_path, # 相对路径
        os.path.join(current_dir, csv_path), # 绝对路径1
        os.path.join(current_dir, 'logs', 'full_evaluation_results.csv'), # 绝对路径2
        os.path.join(os.path.dirname(current_dir), 'logs', 'full_evaluation_results.csv') # 父级目录的logs
    ]
    
    found_path = None
    for p in possible_paths:
        if os.path.exists(p):
            found_path = p
            break
            
    if not found_path:
        print(f"❌ 错误：在以下路径均未找到数据文件:\n{possible_paths}")
        return pd.DataFrame()

    print(f"📂 正在读取数据: {found_path}")

    # 读取 CSV (带编码容错)
    try:
        df = pd.read_csv(found_path, encoding='utf-8')
    except UnicodeDecodeError:
        try:
            df = pd.read_csv(found_path, encoding='gbk')
        except Exception as e:
            print(f"❌ 读取失败: {e}")
            return pd.DataFrame()

    # 解析 JSON 字符串
    for col in ['daily_actions', 'daily_incomes']:
        if col in df.columns:
            df[col] = df[col].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
    
    # 确保索引是数值
    if 'case_study_index' in df.columns:
        df['case_study_index'] = pd.to_numeric(df['case_study_index'], errors='coerce')

    print(f"✅ 成功加载 {len(df)} 条记录。")
    return df

# -----------------------------------------------------------------------------
# 2. 指标计算函数
# -----------------------------------------------------------------------------
def calculate_capture_speed(daily_actions, movie_index, threshold=0.20, max_days=20):
    """
    计算捕获速度：第几天排片占比首次 >= threshold
    """
    if pd.isna(movie_index): return np.nan
    movie_index = int(movie_index)
    
    actions = daily_actions[:max_days]
    for day, schedule in enumerate(actions):
        if len(schedule) > movie_index:
            if schedule[movie_index] >= threshold:
                return day + 1 
    return np.nan

# -----------------------------------------------------------------------------
# 3. 主分析逻辑
# -----------------------------------------------------------------------------
def analyze_and_plot():
    df = load_data()
    if df.empty:
        print("⚠️ 数据为空，无法绘图。请先运行 main.py 生成数据。")
        return

    # --- [关键修复] 确保保存目录存在并使用绝对路径 ---
    try:
        current_dir = os.path.dirname(os.path.abspath(__file__))
    except NameError:
        current_dir = os.getcwd()

    plots_dir = os.path.join(current_dir, 'plots', 'black_horse_analysis')
    os.makedirs(plots_dir, exist_ok=True)
    print(f"📂 图片将保存至: {plots_dir}")
    
    # ========================= [ 34部 黑马电影完整名单 ] =========================
    DARK_HORSE_MOVIES = [
        '流浪地球', '熊出没·原始时代', '新喜剧之王',
        '刺杀小说家', '人潮汹涌', '熊出没·狂野大陆',
        '这个杀手不太冷静', '熊出没·重返地球', '狙击手', '四海',
        '熊出没·伴我“熊芯”', '无名', '深海', '第二十条', '熊出没·逆转时空',
        '我和我的父辈', '坚如磐石', '前任4：英年早婚', '志愿军：雄兵出击', '莫斯科行动',
        '悬崖之上', '人生路不熟', '末路狂花钱', '九龙城寨之围城',
        '封神第一部：朝歌风云', '长安三万里', '学爸', '异形：夺命舰',
        '比悲伤更悲伤的故事', '我的姐姐', '扬名立万', '金手指', '周处除三害', '你想活出怎样的人生'
    ]
    # =========================================================================

    unique_cases = df[['start_date', 'case_study_movie']].drop_duplicates()
    all_results = []

    print(f"\n🚀 开始分析与绘图，共发现 {len(unique_cases)} 个案例...")
    print("=" * 80)

    # 开启交互模式，尝试让图片显示出来
    plt.ion() 

    for _, row in unique_cases.iterrows():
        date = row['start_date']
        movie = row['case_study_movie']
        
        if movie not in DARK_HORSE_MOVIES:
            continue
            
        case_df = df[(df['start_date'] == date) & (df['case_study_movie'] == movie)].copy()
        
        if case_df.empty:
            continue
            
        print(f"🎨 正在绘图: 《{movie}》...")

        # --- 1. 计算捕获速度 ---
        metric_name = "capture_speed_20d"
        threshold = 0.20 
        
        case_df[metric_name] = case_df.apply(
            lambda r: calculate_capture_speed(r['daily_actions'], r['case_study_index'], threshold), 
            axis=1
        )
        
        summary = case_df.groupby('policy')[metric_name].mean()
        summary.name = movie
        all_results.append(summary)

        # --- 2. 绘图 ---
        fig, ax = plt.subplots(figsize=(12, 7))
        
        policies = case_df['policy'].unique()
        styles = ['-', '--', '-.', ':']
        
        for i, policy in enumerate(policies):
            run = case_df[case_df['policy'] == policy].iloc[0]
            idx = run['case_study_index']
            
            if pd.isna(idx): continue
            idx = int(idx)
            
            actions = run['daily_actions']
            if not actions or len(actions) == 0: continue
            
            y_values = []
            for day_act in actions[:20]:
                if len(day_act) > idx:
                    y_values.append(day_act[idx] * 100) 
                else:
                    y_values.append(0)
            
            x_values = range(1, len(y_values) + 1)
            
            if "SAC" in policy:
                c = '#d62728' # 红色
                ls = '-'      # 实线
                lw = 3.5      # 加粗
                alpha = 1.0
                zorder = 10   
            else:
                c = None      
                ls = styles[i % len(styles)]
                lw = 1.5
                alpha = 0.7
                zorder = 2
            
            ax.plot(x_values, y_values, label=policy, color=c, linestyle=ls, linewidth=lw, alpha=alpha, zorder=zorder)

        ax.axhline(y=threshold*100, color='gray', linestyle='--', alpha=0.5, label=f'捕获阈值 {threshold*100:.0f}%')
        
        ax.set_title(f"黑马电影《{movie}》排片趋势响应曲线", fontsize=16)
        ax.set_xlabel("上映天数 (Day)", fontsize=12)
        ax.set_ylabel("排片占比 (%)", fontsize=12)
        ax.set_xlim(1, 20)
        ax.set_xticks(range(1, 21))
        ax.set_ylim(bottom=-2)
        
        ax.grid(True, linestyle=':', alpha=0.6)
        ax.legend(loc='upper left')
        
        # --- 保存图片 ---
        safe_name = movie.replace('·', '').replace('：', '').replace(':', '').replace(' ', '')
        filename = f"{safe_name}_排片趋势.png"
        save_path = os.path.join(plots_dir, filename)
        
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        
        # 关闭图形，避免弹窗过多卡死
        plt.close(fig) 

    print("-" * 60)
    print(f"✅ 所有排片趋势图已保存至:\n   {plots_dir}")
    
    # --- 3. 打印最终速度汇总表 ---
    if all_results:
        print("\n" + "="*80)
        print("### 黑马捕获速度汇总 (达到20%排片所需天数) ###")
        print("数值越小越好，'-' 代表20天内未达标")
        print("="*80)
        
        res_df = pd.concat(all_results, axis=1)
        
        # 排序策略
        preferred_order = ["RL (SAC)", "效率启发式策略", "贪婪启发式策略", "静态启发式策略"]
        existing_order = [p for p in preferred_order if p in res_df.index]
        remaining = [p for p in res_df.index if p not in existing_order]
        final_order = existing_order + remaining
        
        res_df = res_df.loc[final_order]

        print(res_df.T.to_string(float_format="{:.1f}".format, na_rep="-"))
        
        print("\n=== 平均捕获天数 (越小越好) ===")
        print(res_df.mean(axis=1).sort_values().to_string(float_format="{:.2f}".format))

if __name__ == "__main__":
    analyze_and_plot()

⚠️ 检测到交互式环境，使用当前工作目录: D:\App\Pycharm\1118MovieSchedulingSimulation\jupyters
📂 正在读取数据: D:\App\Pycharm\1118MovieSchedulingSimulation\logs\full_evaluation_results.csv
✅ 成功加载 132 条记录。
📂 图片将保存至: D:\App\Pycharm\1118MovieSchedulingSimulation\jupyters\plots\black_horse_analysis

🚀 开始分析与绘图，共发现 33 个案例...
🎨 正在绘图: 《流浪地球》...
🎨 正在绘图: 《比悲伤更悲伤的故事》...
🎨 正在绘图: 《刺杀小说家》...
🎨 正在绘图: 《人潮汹涌》...
🎨 正在绘图: 《熊出没·狂野大陆》...
🎨 正在绘图: 《我的姐姐》...
🎨 正在绘图: 《悬崖之上》...
🎨 正在绘图: 《我和我的父辈》...
🎨 正在绘图: 《扬名立万》...
🎨 正在绘图: 《这个杀手不太冷静》...
🎨 正在绘图: 《熊出没·重返地球》...
🎨 正在绘图: 《狙击手》...
🎨 正在绘图: 《四海》...
🎨 正在绘图: 《熊出没·伴我“熊芯”》...
🎨 正在绘图: 《无名》...
🎨 正在绘图: 《深海》...
🎨 正在绘图: 《人生路不熟》...
🎨 正在绘图: 《长安三万里》...
🎨 正在绘图: 《封神第一部：朝歌风云》...
🎨 正在绘图: 《学爸》...
🎨 正在绘图: 《坚如磐石》...
🎨 正在绘图: 《前任4：英年早婚》...
🎨 正在绘图: 《志愿军：雄兵出击》...
🎨 正在绘图: 《莫斯科行动》...
🎨 正在绘图: 《金手指》...
🎨 正在绘图: 《第二十条》...
🎨 正在绘图: 《熊出没·逆转时空》...
🎨 正在绘图: 《周处除三害》...
🎨 正在绘图: 《你想活出怎样的人生》...
🎨 正在绘图: 《末路狂花钱》...
🎨 正在绘图: 《九龙城寨之围城》...
🎨 正在绘图: 《异形：夺命舰》...
------------------------------------------------------------
✅ 所有排片趋势图已保存至:
   